<a href="https://colab.research.google.com/github/urvesh2012/MLTSA25_UBhagat/blob/main/labs/TimeSeriesClassificationWithKerasTransformers_ClassCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
from keras import layers

In [2]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

input_shape = x_train.shape[1:]

In [3]:
x_train.shape

(3601, 500, 1)

In [4]:
def train_and_evaluate(model, Xtrn, Xtst, Ytrn, Ytst):
    cb_early = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    model.fit(
        Xtrn,
        Ytrn,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[cb_early]
    )
    loss_tst, acc_tst = model.evaluate(Xtst, Ytst, verbose=1)
    print(f"Loss calculated on the testing set: {loss_tst:.4f}")
    print(f"Accuracy calculated on the testing set: {acc_tst:.4f}")
    return None

In [5]:
def transformer_block(
    x,
    input_shape,
    heads=4,
    key_dim=256,
    encoder_proj_dim=4,
    dropout_attention=0.25,
    dropout_projection=0.25,
):
    x0 = layers.MultiHeadAttention(num_heads=heads,key_dim=key_dim)(x,x)
    x0 = layers.Dropout(dropout_attention)(x0)
    x0 = layers.Add()([x,x0])  # The "residual Connection"
    x0 = layers.LayerNormalization()(x0)

    x1 = layers.Conv1D(filters=encoder_proj_dim,kernel_size=1,activation="relu")(x0)
    x1 = layers.Conv1D(filters=input_shape[-1],kernel_size=1, activation="relu")(x1)
    x1 = layers.Dropout(dropout_projection)(x1)
    x1 = layers.Add()([x0,x1])
    x1 = layers.LayerNormalization()(x1)

    return x1


def build_transformer_model(
    input_shape,
    encoder_blocks=0,
    feed_forward_units=[128],
):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    for _ in range(encoder_blocks):
        x = transformer_block(
            x,
            input_shape
        )

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for n in feed_forward_units:
        x = layers.Dense(n, activation="relu")(x)
        x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(2,activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    return model


transformer_model = build_transformer_model(input_shape)
transformer_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
transformer_model.summary()

train_and_evaluate(transformer_model, x_train, x_test, y_train, y_test)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 500, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 500, 1)    │      7,169 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 500, 1)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 500, 1)    │          0 │ input_layer[0][0… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 500, 1)    │          2 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 500, 4)    │          8 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 500, 1)    │          5 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 500, 1)    │          0 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 500, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 500, 1)    │          2 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 500, 1)    │      7,169 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 500, 1)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 500, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 500, 1)    │          2 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 500, 4)    │          8 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 500, 1)    │          5 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 500, 1)    │          0 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 500, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 500, 1)    │          2 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 93,130 (363.79 KB)

 Trainable params: 93,130 (363.79 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 39s 193ms/step - loss: 0.6931 - sparse_categorical_accuracy: 0.5272 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5118
Epoch 2/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 13s 142ms/step - loss: 0.6931 - sparse_categorical_accuracy: 0.5120 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5118
Epoch 3/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 139ms/step - loss: 0.6931 - sparse_categorical_accuracy: 0.4999 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5118
Epoch 4/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 139ms/step - loss: 0.6930 - sparse_categorical_accuracy: 0.5172 - val_loss: 0.6930 - val_sparse_categorical_accuracy: 0.5118
Epoch 5/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 21s 143ms/step - loss: 0.6929 - sparse_categorical_accuracy: 0.5209 - val_loss: 0.6930 - val_sparse_categorical_accuracy: 0.5118
Epoch 6/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 143ms/step - loss: 0.6930 - sparse_categorical_accuracy: 0.5125 - val_loss: 0.6930 - val_sparse_categorical_accuracy:

In [ ]:
def build_ff_model(
    input_shape,
    feed_forward_units=[128]
):
    inputs = ...
    x = inputs

    x = ...
    for ... in ...:
        x = ...
        x = ...

    outputs = ...
    model = keras.Model(..., ...)

    return model


input_shape = x_train.shape[1:]
ff_model = build_ff_model(input_shape)
ff_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=...),
    metrics=["sparse_categorical_accuracy"],
)
ff_model.summary()

train_and_evaluate(ff_model, x_train, x_test, y_train, y_test)